In [1]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [2]:
import itertools
import os

import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import teotil3 as teo
from tqdm.notebook import tqdm

plt.style.use("ggplot")

# Run model

In [3]:
# Period of interest
st_yr, end_yr = 2017, 2019

# Vassdragsområder of interest
vassom_list = range(1, 18)

# Data paths
oslomod_scen_dir = r"/home/jovyan/shared/common/oslofjord_modelling/phase3_scenarios"

# Scenarios
scen_list = ["Baseline", "Scenario_A", "Scenario_B"]

## 1. Run TEOTIL3

In [4]:
input_data_fold = os.path.join(oslomod_scen_dir, "teotil3_input_files")
result_csv = os.path.join(
    oslomod_scen_dir, f"teotil3_oslomod_results_{st_yr}-{end_yr}.csv"
)

df_list = []
years = range(st_yr, end_yr + 1)
for scen in scen_list:
    print("Processing:", scen)
    for year in years:
        csv_path = os.path.join(
            input_data_fold,
            scen,
            f"oslomod_teotil3_input_data_{scen.lower()}_{year}.csv",
        )
        g = teo.model.run_model(
            csv_path,
            id_col="regine",
            next_down_col="regine_down",
            totals_from_subfracs=True,
        )
        res_df = teo.model.model_to_dataframe(
            g, id_col="regine", next_down_col="regine_down"
        )
        res_df["scenario"] = scen
        res_df["year"] = year
        df_list.append(res_df)
mod_df = pd.concat(df_list, axis="rows")
id_cols = ["scenario", "year", "regine", "regine_down"]
cols = [col for col in mod_df.columns if col not in id_cols]
mod_df = mod_df[id_cols + cols]
mod_df.to_csv(result_csv, index=False)

Processing: Baseline
Processing: Scenario_A
Processing: Scenario_B
